In [17]:
import pandas as pd
import torch.utils.data as tdata
import numpy as np
import pykitti

import util

In [5]:
basedir = 'dataset/'
sequence = '00'

In [37]:
class OdometryDataset(tdata.Dataset):
    def __init__(self, basedir, sequence, n_points):
        self._data = pykitti.odometry(basedir, sequence)
        self._velo = list(self._data.velo)
        self._poses = self._data.poses
        self._n_points = n_points
    
    def __getitem__(self, i):
        a_cloud = self._velo[i][:self._n_points]
        b_cloud = self._velo[i+1][:self._n_points]
        a_R = self._poses[i][:3, :3]
        a_T = self._poses[i][:3, 3]
        b_R = self._poses[i+1][:3, :3]
        b_T = self._poses[i+1][:3, 3]
        
        R_ab = b_R - a_R
        R_ba = -R_ab
        E_ab = util.npmat2euler(R_ab.reshape((1, 3, 3)), seq='xyz')
        E_ba = util.npmat2euler(R_ba.reshape((1, 3, 3)), seq='xyz')
        T_ab = b_T - a_T
        T_ba = -T_ab
        
        a_cloud = a_cloud.T[:3, :]
        b_cloud = b_cloud.T[:3, :]
        
        return a_cloud, b_cloud, R_ab, T_ab, R_ba, T_ba, E_ab.flatten(), E_ba.flatten()
        
    
    def __len__(self):
        return len(self._data) - 1
        

In [38]:
dataset = OdometryDataset(basedir, sequence, 1000)

In [40]:
for c in dataset[3]:
    print(c)
    print()

[[ 69.86689     69.464485    69.461075   ... -44.651432   -44.70857
  -44.749844  ]
 [  0.5877604    0.8026724    1.0205835  ...  -0.32381812  -0.39586148
   -0.53787684]
 [  2.57187      2.5578227    2.5577748  ...   1.7166728    1.7186664
    1.7196941 ]]

[[ 69.12951     68.6691      68.69989    ... -50.025566   -50.12413
  -47.027775  ]
 [  0.25589645   0.4698096    0.5777659  ...   0.71103007   0.5561565
    0.37514564]
 [  2.5469437    2.5308967    2.5318732  ...   1.9065845    1.9096221
    1.8028474 ]]

[[-1.590000e-05  5.120050e-04 -2.064927e-03]
 [-5.287120e-04 -5.600000e-06 -1.153120e-03]
 [ 2.060695e-03  1.160670e-03 -1.960000e-05]]

[-0.0468429  -0.02836258  0.857684  ]

[[ 1.590000e-05 -5.120050e-04  2.064927e-03]
 [ 5.287120e-04  5.600000e-06  1.153120e-03]
 [-2.060695e-03 -1.160670e-03  1.960000e-05]]

[ 0.0468429   0.02836258 -0.857684  ]

[ 179.99913     -0.4727446 -179.99808  ]

[-0.2646417   0.4730122   0.11816004]

